In [28]:
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM

from rich import print as rprint
from rich.traceback import install
install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7fe74a23edf0>>

In [372]:
import importlib
import tokenizer as t
t = importlib.reload(t) 

In [337]:
DATA_DIR = "./data/"
DATA_DIR_TRAIN = DATA_DIR + "wikitext-2/wiki.train.tokens"
DATA_DIR_VAL = DATA_DIR + "wikitext-2/wiki.valid.tokens"
DATA_DIR_TEST = DATA_DIR + "wikitext-2/wiki.test.tokens"
TOKENIZER_PATH = "gpt2"
SAVE_DATA_DIR = "./data/tokenized_gpt.pkl"

In [373]:
t.tokenize(TOKENIZER_PATH, DATA_DIR_TRAIN, DATA_DIR_VAL, DATA_DIR_TEST, SAVE_DATA_DIR, pretrained=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (3084 > 1024). Running this sequence through the model will result in indexing errors


In [24]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [351]:
DATA_DIR = './data/'
DATA_DIR_TRAIN = DATA_DIR + 'wikitext-2/wiki.train.tokens'
with open(DATA_DIR_TRAIN) as f:
    entries_train = f.read().split(" \n \n = ")
a = tokenizer(entries_train)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-351-79758360ff6c>:5 in <module>                                            │
│                                                                                           │
│ /home/tiankang/software/anaconda3/lib/python3.8/site-packages/transformers/tokenization_u │
│ tils_base.py:2344 in encode_plus                                                          │
│                                                                                           │
│   2341 │   │   │   **kwargs,                                                              │
│   2342 │   │   )                                                                          │
│   2343 │   │                                                                              │
│ ❱ 2344 │   │   return self._encode_plus(                                                  │
│   2345 │   │   │   text=text,                                                             │
│   2346 │   │   │   text_pair=text_pair,                                                   │
│   2347 │   │   │   add_special_tokens=add_special_tokens,                                 │
│                                                                                           │
│ /home/tiankang/software/anaconda3/lib/python3.8/site-packages/transformers/models/gpt2/to │
│ kenization_gpt2_fast.py:173 in _encode_plus                                               │
│                                                                                           │
│   170 │   │   │   "to use it with pretokenized inputs."                                   │
│   171 │   │   )                                                                           │
│   172 │   │                                                                               │
│ ❱ 173 │   │   return super()._encode_plus(*args, **kwargs)                                │
│   174 │                                                                                   │
│   175 │   def save_vocabulary(self, save_directory: str, filename_prefix: Optional[str] = │
│   176 │   │   files = self._tokenizer.model.save(save_directory, name=filename_prefix)    │
│                                                                                           │
│ /home/tiankang/software/anaconda3/lib/python3.8/site-packages/transformers/tokenization_u │
│ tils_fast.py:458 in _encode_plus                                                          │
│                                                                                           │
│   455 │   ) -> BatchEncoding:                                                             │
│   456 │   │                                                                               │
│   457 │   │   batched_input = [(text, text_pair)] if text_pair else [text]                │
│ ❱ 458 │   │   batched_output = self._batch_encode_plus(                                   │
│   459 │   │   │   batched_input,                                                          │
│   460 │   │   │   is_split_into_words=is_split_into_words,                                │
│   461 │   │   │   add_special_tokens=add_special_tokens,                                  │
│                                                                                           │
│ /home/tiankang/software/anaconda3/lib/python3.8/site-packages/transformers/models/gpt2/to │
│ kenization_gpt2_fast.py:163 in _batch_encode_plus                                         │
│                                                                                           │
│   160 │   │   │   "to use it with pretokenized inputs."                                   │
│   161 │   │   )                                                                           │
│   162 │   │                                                                               │
│ ❱ 163 │   │   return super()._batch_encode_plus(*a

In [29]:
model.eval()
None

In [287]:
test_sample = ['The name of the president of the United States is']
encoded_input = tokenizer(test_sample, return_tensors='pt')

In [319]:
encoded_input

{'input_ids': tensor([[ 464, 1438,  286,  262, 1893,  286,  262, 1578, 1829,  318]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [288]:
encoded_input.input_ids.shape

torch.Size([1, 10])

In [289]:
with torch.no_grad():
    output = model(**encoded_input)

In [290]:
dir(output)

['__annotations__',
 '__class__',
 '__contains__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'attentions',
 'clear',
 'copy',
 'cross_attentions',
 'fromkeys',
 'get',
 'hidden_states',
 'items',
 'keys',
 'logits',
 'loss',
 'move_to_end',
 'past_key_values',
 'pop',
 'popitem',
 'setdefault',
 'to_tuple',
 'update',
 'values']

In [291]:
output.logits.shape

torch.Size([1, 10, 50257])

In [292]:
len(output.past_key_values), len(output.past_key_values[0]), output.past_key_values[0][1].shape, output.past_key_values[0][0].shape  # [batch_size, num_heads, seq_length, emb_size_per_head]

(12, 2, torch.Size([1, 12, 10, 64]), torch.Size([1, 12, 10, 64]))

In [293]:
tokenizer.decode(output.logits[:, -1, :].argmax(dim=1)[0])

' not'

In [257]:
output.past_key_values[0][0].shape, output.past_key_values[0][1].shape, 

(torch.Size([1, 12, 8, 64]), torch.Size([1, 12, 8, 64]))

In [308]:
from datamodule import WikiText2DataModule

from tqdm.auto import tqdm

In [374]:
DATA_DIR = './data/tokenized_gpt.pkl'
BATCH_SIZE = 128
BPTT_TRAIN = 64
BPTT_VAL = 64
datamodule = WikiText2DataModule(DATA_DIR, BATCH_SIZE, BPTT_TRAIN, BPTT_VAL)
datamodule.setup(stage='fit')
loader = datamodule.val_dataloader()

In [375]:
device = 'cuda:9'
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
model.eval()
losses = []
seq_lengths = []

In [376]:
with torch.no_grad():
    for batch in tqdm(loader):
        input_idxs, target_idxs = batch
        # rprint(target_idxs.shape)
        input_idxs, target_idxs = input_idxs.to(device), target_idxs.to(device)  # both: [seq_length, batch_size]
        logits = model(input_idxs).logits  # [seq_length, batch_size, vocab_size]
        seq_length = input_idxs.shape[0]
        loss = criterion(logits.flatten(end_dim=1), target_idxs.flatten())
        losses.append(loss)
        seq_lengths.append(seq_length)

In [314]:
total_loss = sum([loss * seq_length for loss, seq_length in zip(losses, seq_lengths)])
total_len = sum(seq_lengths)
torch.exp(total_loss / total_len)

tensor(9649.8438, device='cuda:9')